In [1]:
import numpy as np
import scipy.stats

In [2]:
import tensorflow as tf
from numba import jit

ImportError: Traceback (most recent call last):
  File "/data/anaconda3/lib/python3.5/site-packages/tensorflow/python/__init__.py", line 61, in <module>
    from tensorflow.python import pywrap_tensorflow
  File "/data/anaconda3/lib/python3.5/site-packages/tensorflow/python/pywrap_tensorflow.py", line 28, in <module>
    _pywrap_tensorflow = swig_import_helper()
  File "/data/anaconda3/lib/python3.5/site-packages/tensorflow/python/pywrap_tensorflow.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow', fp, pathname, description)
  File "/data/anaconda3/lib/python3.5/imp.py", line 242, in load_module
    return load_dynamic(name, filename, file)
  File "/data/anaconda3/lib/python3.5/imp.py", line 342, in load_dynamic
    return _load(spec)
ImportError: /lib64/libc.so.6: version `GLIBC_2.16' not found (required by /data/anaconda3/lib/python3.5/site-packages/tensorflow/python/_pywrap_tensorflow.so)


Failed to load the native TensorFlow runtime.

See https://

In [3]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [4]:
@jit
def f(theta, x, returngrad=False):
    fval = (theta[0])*(theta[1] - x)
    if returngrad==False:
        return fval
    else:
        if (np.isscalar(x)):
            xshape = ()
        else:
            xshape = x.shape
        dfdtheta = np.zeros(theta.shape + xshape)
        # want this to broadcast correctly regardless of whether x is a matrix or vector
        dfdtheta[0,...] = fval/theta[0]
        dfdtheta[1,...] = theta[0]
        dfdtheta[2,...] = 0
        return fval, dfdtheta
    
@jit
def g(theta, x, returngrad=False):
    if (np.isscalar(x)):
        xshape = ()
        gval = theta[2]
    else:
        xshape = x.shape
        gval = np.ones(xshape)*theta[2]
        
    if returngrad==False:
        return gval
    else:
        dgdtheta = np.zeros(theta.shape + xshape)
        # want this to broadcast correctly regardless of whether x is a matrix or vector
        dgdtheta[0,...] = 0
        dgdtheta[1,...] = 0
        dgdtheta[2,...] = 1
        return gval, dgdtheta

In [5]:
# create the sparse A and B tensors
# this is a pure Python and Numpy function
# and therefore can be accelerated with Numba's JIT compiler
@jit
def createAB(M, k, h, tol=np.inf):
    AMtheta = 0.5
    omtheta = 1.-AMtheta
    alpha = np.zeros(2)
    alpha[0] = 0.5/(AMtheta*omtheta)
    alpha[1] = alpha[0]*(omtheta*omtheta + AMtheta*AMtheta)
    grid = np.arange(-M,(M+1))*k
    # N = 2*M+1
    N = len(grid)
    outAval = np.zeros(1)
    outAind = np.zeros((1,3),dtype='int64')
    Aval = [outAval]
    Aind = [outAind]
    th = np.array([1.,0.5,0.75])
    fvec = f(th, grid)
    gvec = g(th, grid)
    g2m1 = np.square(gvec)
    col1 = np.ones((N,1),dtype='int64')
    colr = np.array([np.arange(0,N,dtype='int64')]).T
    for i in range(N):
        for j in range(N):
            if (abs(i-j) > tol):
                continue

            yi = grid[i]
            ystar = grid[j]
            mv = ystar + (alpha[0]*f(th,ystar) - alpha[1]*fvec)*omtheta*h
            g2star = np.square(g(th,ystar))
            vv = np.maximum(0, alpha[0]*g2star - alpha[1]*g2m1)*omtheta*h
            thesevals = scipy.stats.norm.pdf(x=yi, loc=mv, scale=np.sqrt(vv))
            indblock = np.hstack([i*col1,j*col1,colr])
            Aind.append(indblock)
            Aval.append(thesevals)
    
    outAind = np.vstack(Aind[1:])
    outAval = np.hstack(Aval[1:])

    Bds = [N,N]
    Bind = np.zeros((0,2))
    thisvar = g2m1*AMtheta*h
    c0mod = 1./np.sqrt(2.0*np.pi*thisvar)
    propvals = np.exp(-(AMtheta*h/2.0)*np.square(fvec)/g2m1) * c0mod
    Bval = [propvals]
    indblock = np.hstack([colr, colr])
    Bind = [np.vstack([Bind, indblock])]

    for curdiag in np.arange(1,N):
        thislen = N - curdiag
        mymean = curdiag*k + fvec*h*AMtheta
        thisdiag = np.exp(-np.square(mymean)/(2.0*thisvar)) * c0mod
        thisdiag = thisdiag[-thislen:]
        indblock = np.hstack([colr[:(N-curdiag)],(colr[:(N-curdiag)]+curdiag)])
        Bind.append(indblock) 
        Bval.append(thisdiag) 

    for curdiag in np.arange(1,N):
        thislen = N - curdiag
        mymean = -curdiag*k + fvec*h*AMtheta
        thisdiag = np.exp(-np.square(mymean)/(2.0*thisvar)) * c0mod
        thisdiag = thisdiag[:thislen]
        indblock = np.hstack([(colr[:(N-curdiag)]+curdiag),colr[:(N-curdiag)]])
        Bind.append(indblock) 
        Bval.append(thisdiag) 

    outBind = np.vstack(Bind)
    outBval = np.hstack(Bval)
    
    return outAind, outAval, outBind, outBval

In [6]:
# create the Anderon-Mattingly propagator
def computeAMprop(Aind, Aval, Bden):
    sortAind = np.argsort(Aind[:,2])
    newAind = Aind[sortAind,:]
    newAval = np.array(Aval)
    newAval = newAval[sortAind]
    mypropcols = []
    startind = 0
    thisds = [N,N]
    for j in range(N):
        if j==(N-1):
            endind = len(Aval)
        else:
            endind = np.searchsorted(newAind[:,2],(j+1))

        thesevals = newAval[startind:endind]
        theseinds = newAind[startind:endind,0:2]
        thisten = tf.SparseTensor(indices=theseinds,values=thesevals,dense_shape=thisds)
        newcolumn = k*tf.sparse_tensor_dense_matmul(thisten, tf.slice(Bden, [0,j], [N,1]))
        mypropcols.append(newcolumn[:,0])
        startind = endind

    myprop = tf.stack(mypropcols)
    return myprop


In [12]:
# wasteful way where we first compute a massive tensor dot product and then essentially pull out diagonals
# Ads = [N,N,N]
# Aten = tf.SparseTensor(indices=Aind,values=Aval,dense_shape=Ads)
# propten = k*tf.tensordot(tf.sparse_tensor_to_dense(Aten,validate_indices=False), Bden, axes = [[1], [0]])
# propden = sess.run(propten)
# actprop = np.zeros((N,N))
# for j in range(N):
#     actprop[j] = np.diag(propden[j])

In [11]:
M = 20
N = 2*M+1
k = 0.20
h = 0.5
Aind, Aval, Bind, Bval = createAB(M, k, h)
Bds = [N, N]
Bten = tf.SparseTensor(indices=Bind,values=Bval,dense_shape=Bds)
Bden = tf.sparse_tensor_to_dense(Bten,validate_indices=False)
prop = computeAMprop(Aind, Aval, Bden)

# check normalization
sess.run(tf.reduce_sum(prop,axis=1))*k
npprop = sess.run(prop)

In [12]:
np.set_printoptions(precision=2)
print(npprop[:,0])

[  2.75e-04   8.12e-05   2.17e-05   5.26e-06   1.16e-06   2.36e-07
   4.35e-08   7.36e-09   1.14e-09   1.61e-10   2.08e-11   2.47e-12
   2.67e-13   2.65e-14   2.41e-15   2.00e-16   1.52e-17   1.05e-18
   6.70e-20   3.90e-21   2.07e-22   1.01e-23   4.50e-25   1.83e-26
   6.84e-28   2.33e-29   7.29e-31   2.08e-32   5.44e-34   1.30e-35
   2.85e-37   5.70e-39   1.04e-40   1.75e-42   2.68e-44   3.76e-46
   4.83e-48   5.67e-50   6.09e-52   5.99e-54   5.39e-56]
